<a href="https://colab.research.google.com/github/MikChristensen/Algo-Trading-Examples/blob/main/ML_Strategies_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM

https://github.com/soms98/Stock-Price-Prediction-Time-Series-LSTM-Model-Keras-Tensorflow/blob/master/HDFC.ipynb
https://www.youtube.com/watch?v=CbTU92pbDKw&t=1312s

https://medium.com/mlearning-ai/multivariate-time-series-forecasting-using-rnn-lstm-8d840f3f9aa7



In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf

# Import tensorflow packages
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import EarlyStopping # EarlyStopping during model training
from sklearn.metrics import mean_absolute_error, mean_squared_error # Packages for measuring model performance / errors
from sklearn.model_selection import TimeSeriesSplit

# Displaying data
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

# Preprocessing data
from talib import abstract
from sklearn.preprocessing import RobustScaler, MinMaxScaler
import math # Mathematical functions


ModuleNotFoundError: No module named 'talib'

In [ ]:
!pip install ta-lib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 368.5/368.5 kB 8.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for ta-lib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for ta-lib
Failed to build ta-lib
ERROR: Could not build wheels for ta-lib, which is required to install pyproject.toml-based projects


## Project variabels

In [ ]:
symbol = "EURUSD=X"
interval ='5m'
period = '1mo'

train_size_fraction = 0.70

## Download data

In [ ]:
def getHistoryYfinance(symbol, interval, period):

    data = yf.download(  # or pdr.get_data_yahoo(...
    # Yahoo finance symbol
    tickers = symbol,

    # use "period" instead of start/end
    # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
    # (optional, default is '1mo')
    period = period,

    # fetch data by interval (including intraday if period < 60 days)
    # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
    # (optional, default is '1d')
    interval = interval,

    # group by ticker (to access via data['SPY'])
    # (optional, default is 'column')
    group_by = 'ticker',

    # adjust all OHLC automatically
    # (optional, default is False)
    auto_adjust = False,

    # download pre/post regular market hours data
    # (optional, default is False)
    prepost = False,

    # use threads for mass downloading? (True/False/Integer)
    # (optional, default is True)
    threads = True,

    # proxy URL scheme use use when downloading?
    # (optional, default is None)
    proxy = None
    )

    data["returns"] = data['Close'] / data['Close'].shift(1)
    data.drop(['Adj Close', 'Volume'], axis='columns', inplace=True)
    data.dropna(inplace=True)

    return data

#Data_Raw = getHistoryYfinance(symbol, interval, period)
#Data_Raw.head()

In [ ]:
def getHistoryOanda(pair = "EUR_USD", period = 'M5'):
    #columns = ['volume',	'mid_o',	'mid_h',	'mid_l',	'mid_c', 'time']
    columns = ['mid_o',	'mid_h',	'mid_l',	'mid_c', 'time']

    if(period == 'H1'):
        data = pd.read_pickle(f"{pair}_H1.pkl")

    if(period == 'M5'):
        data = pd.read_pickle(f"{pair}_M5.pkl")

    data = data[columns]

    #dict = {"time" : "Datetime",  "volume" : "Volume" , "mid_o" : "Open" , "mid_h" : "High", "mid_l" : "Low" , "mid_c" : "Close"}
    dict = {"time" : "Datetime", "mid_o" : "Open" , "mid_h" : "High", "mid_l" : "Low" , "mid_c" : "Close"}

    data = data.rename(columns= dict, inplace=False)

    data["returns"] = data["Close"] / data["Close"].shift(1) - 1

    data = data.dropna()
    data = data.set_index("Datetime")
    return data


Data_Raw = getHistoryOanda()
Data_Raw.head()


## Display data

In [ ]:
plt.figure(figsize=(14,5))
sns.set_style("ticks")
sns.lineplot(data=Data_Raw,x="Datetime",y='Close',color='blue')
sns.despine()
plt.title(symbol + ' Close Price',size='x-large',color='black')

## Create features

In [ ]:
def prepare_features(data):
    ''' Prepares the feature columns for training set and test set.
    '''
    feature_columns = []
    lags = 5

    for lag in range(1, lags + 1):
        col = "lag{}".format(lag)
        data[col] = data["returns"].shift(lag)
        feature_columns.append(col)

    data['Ask_Dir'] = np.where(data['Close'].shift(-1) > data.Close, 1, 0)
    feature_columns.append('Ask_Dir')

    data['SMA'] = abstract.SMA(data['Close'], timeperiod=12)
    feature_columns.append('SMA')
    data['SMA_Dir'] = np.where(data['SMA'].shift(-1) > data.SMA, 1, 0)
    feature_columns.append('SMA_Dir')

    data['RSI'] = abstract.RSI(data['Close'], timeperiod=12)
    feature_columns.append('RSI')
    data['RSI_Dir'] = np.where(data['RSI'].shift(-1) > data.RSI, 1, 0)
    feature_columns.append('RSI_Dir')

    data['fastk'], data['fastd'] = abstract.STOCHRSI(data["Close"], timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)
    feature_columns.append('fastk')
    feature_columns.append('fastd')
    data['fastk_Dir'] = np.where(data['fastk'].shift(-1) > data.fastk, 1, 0)
    feature_columns.append('fastk_Dir')
    data['fastd_Dir'] = np.where(data['fastd'].shift(-1) > data.fastd, 1, 0)
    feature_columns.append('fastd_Dir')

    data.dropna(inplace=True)

    return data, feature_columns

Data_Processed, feature_columns = prepare_features(Data_Raw)

In [ ]:
Data_Processed.head()

In [ ]:
Data_Processed.tail()

In [ ]:
Data_Processed.describe()

In [ ]:
Data_Processed.dtypes

In [ ]:
print(feature_columns)

In [ ]:
df = Data_Processed.sort_values(by=['Datetime']).copy()
data = pd.DataFrame(df)
data_filtered = data[feature_columns]

In [ ]:
# We add a prediction column and set dummy values to prepare the data for scaling
data_filtered_ext = data_filtered.copy()
data_filtered_ext['Prediction'] = df['returns']

## Split Data

In [ ]:

min_date = data_filtered_ext.index.min()
max_date = data_filtered_ext.index.max()
print("Min:", min_date, "Max:", max_date)



In [ ]:

train_percent = .75
time_between = max_date - min_date
train_cutoff = min_date + train_percent*time_between
train_cutoff


In [ ]:

df_train = data_filtered_ext[data_filtered_ext.index <= train_cutoff]
df_test = data_filtered_ext[data_filtered_ext.index > train_cutoff]
print("Train:", df_train.index.min(), df_train.index.max())
print("Test:", df_test.index.min(), df_test.index.max())


## Scaling data
### Convert to numpy array

In [ ]:
# Get the number of rows in the data
def convert_to_np(pd_data):
    nrows = pd_data.shape[0]

    # Convert the data to numpy values
    np_data_unscaled = np.array(pd_data)
    np_data = np.reshape(np_data_unscaled, (nrows, -1))
    print(np_data.shape)

    return np_data


In [ ]:
np_x_train = convert_to_np(df_train[feature_columns])
np_x_test = convert_to_np(df_test[feature_columns])
np_y_train = convert_to_np(df_train['Prediction'])
np_y_test = convert_to_np(df_test['Prediction'])

### Scale data

In [ ]:
# Transform the data by scaling each feature to a range between 0 and 1
def scale_np(np_data):
    obj_scaler = MinMaxScaler()
    np_data_scaled = obj_scaler.fit_transform(np_data)

    return np_data_scaled, obj_scaler


In [ ]:
np_x_train_scale, scaler = scale_np(np_x_train)
np_x_test_scale, scaler_data = scale_np(np_x_test)
np_y_train_scale, scaler = scale_np(np_y_train)
np_y_test_scale, scaler_pred = scale_np(np_y_test)

## Split data into training and test

In [ ]:
index_target = df.columns.get_loc("returns")

In [ ]:
# Prediction Index
sequence_length = 5
# The RNN needs data with the format of [samples, time steps, features]
# Here, we create N samples, sequence_length time steps per sample, and 6 features
def partition_x_dataset(sequence_length, data):
    x = []
    data_len = data.shape[0]
    for i in range(sequence_length, data_len):
        x.append(data[i-sequence_length:i,:]) #contains sequence_length values 0-sequence_length * columsn

    # Convert the x to numpy array
    x = np.array(x)

    return x

def partition_y_dataset(sequence_length, data):
    y = []
    data_len = data.shape[0]
    for i in range(sequence_length, data_len):
        y.append(data[i, 0]) #contains the prediction values for validation,  for single-step prediction

    # Convert the y to numpy array
    y = np.array(y)
    return y

# Generate training data and test data
x_train = partition_x_dataset(sequence_length, np_x_train_scale)
x_test = partition_x_dataset(sequence_length, np_x_test_scale)
y_train = partition_y_dataset(sequence_length, np_y_train_scale)
y_test = partition_y_dataset(sequence_length, np_y_test_scale)


In [ ]:
len(x_train)

In [ ]:
# Print the shapes: the result is: (rows, training_sequence, features) (prediction value, )
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [ ]:
# Validate that the prediction value and the input match up
# The last close price of the second input sample should equal the first prediction value
print(x_train[1][sequence_length-1][index_target])
print(y_train[0])

In [ ]:
# Configure the neural network model
model = Sequential()

# Model with n_neurons = inputshape Timestamps, each with x_train.shape[2] variables
#n_neurons = x_train.shape[1] * x_train.shape[2]
n_neurons = 20
print(n_neurons, x_train.shape[1], x_train.shape[2])
model.add(LSTM(n_neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(LSTM(n_neurons, return_sequences=True, activation='relu'))
model.add(LSTM(n_neurons, return_sequences=False))
model.add(Dense(5))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mse')

In [ ]:
# Training the model
#epochs = 50
epochs = 200
batch_size = 16
early_stop = EarlyStopping(monitor='loss', patience=5, verbose=1)
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(x_test, y_test)
                   )

In [ ]:
# Plot training & validation loss values
fig, ax = plt.subplots(figsize=(16, 5), sharex=True)
sns.lineplot(data=history.history["loss"])
plt.title("Model loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
ax.xaxis.set_major_locator(plt.MaxNLocator(epochs))
plt.legend(["Train", "Test"], loc="upper left")
plt.grid()
plt.show()

# Get the predicted values

In [ ]:

y_pred_scaled = model.predict(x_test)

# Unscale the predicted values

In [ ]:
# Unscale the predicted values
y_pred = scaler_pred.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler_pred.inverse_transform(y_test.reshape(-1, 1))

# Mean Absolute Error (MAE)
MAE = mean_absolute_error(y_test_unscaled, y_pred)
print(f'Median Absolute Error (MAE): {np.round(MAE, 2)}')

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print(f'Mean Absolute Percentage Error (MAPE): {np.round(MAPE, 2)} %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled)) ) * 100
print(f'Median Absolute Percentage Error (MDAPE): {np.round(MDAPE, 2)} %')

In [ ]:
df_result = pd.DataFrame(y_test)

df_result['pred'] = y_test_unscaled.tolist()

In [ ]:
df_result

In [ ]:
predictions = model.predict(x_test)
predictions = scaler_pred.inverse_transform(predictions)
rmse = np.sqrt(np.mean(predictions - y_test)**2)
rmse


In [ ]:
data = Data_Raw.filter(['returns'])
train = data[:4420]
validation = data[4420:]
validation['Predictions'] = predictions
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date')
plt.ylabel('Close Price USD ($)')
plt.plot(train)
plt.plot(validation[['returns', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()

In [ ]:
y_test.shape

In [ ]:
y_train.shape

In [ ]:
len(x_train)